# SDEV 140 Fall 2023
# Evan Mitchell
# Rainbow Six Siege App
# Keeps track of current match round scores as well as
# Overall round win/loss for each operator

Classes used to maintain object states
Objects will be saved to .obj files using pickle

In [1]:
from breezypythongui import EasyFrame
import tkinter as tk
import pickle
from PIL import ImageTk,Image

In [2]:
class GameMatch:
    def __init__(self,username):
        self.wins = 0
        self.loses = 0
        self.user = username

In [77]:
class Operator:
    def __init__(self,name,side,wins=0,loses=0):
        self.name = name
        self.wins = wins
        self.loses = loses
        self.side = side

Our MainFrame class will represent our main window with a button to start a new match

In [4]:
class MainFrame(EasyFrame):
    def __init__(self):
        EasyFrame.__init__(self,title="Game",height=500,width=500)
        self.config(bg="blue")

The Match class will handle the majority of our logic

In [78]:


class Match(EasyFrame):

    def __init__(self):
        EasyFrame.__init__(self,title="Match",width=500,height=500)
        self.config(bg="#869eb9")
        self.match = GameMatch("emitchell")
        self.img_label = self.addLabel(text="",row=2,column=1,sticky="NSEW")
        self.display = self.addTextField(text="",row=2,column=0,sticky="NSEW")
        
        ### Operator Setup ###
        self.all_ops = {"tachanka":"defend","castle":"defend","nomad":"attack","sens":"attack","valkyrie":"defend","thatcher":"attack",
                       "tubarao":"defend","ram":"attack","fenrir":"defend","brava":"attack","solis":"defend","grim":"attack",
                       "azami":"defend","thorn":"defend","osa":"attack","thunderbird":"defend","flores":"attack","aruni":"defend",
                       "zero":"attack","ace":"attack","melusi":"defend","oryx":"defend","iana":"attack","wamai":"defend","kali":"attack",
                       "amaru":"attack","goyo":"defend","nokk":"attack","warden":"defend","mozie":"defend","gridlock":"attack","kaid":"defend",
                       "clash":"defend","maverick":"attack","maestro":"defend","alibi":"defend","lion":"attack","finka":"attack",
                       "vigil":"defend","dokkaebi":"attack","zofia":"attack","ela":"defend","ying":"attack","lesion":"defend",
                       "mira":"defend","jackal":"attack","hibana":"attack","echo":"defend","caveira":"defend","capitao":"attack",
                       "blackbeard":"attack","buck":"attack","frost":"defend","mute":"defend","sledge":"attack","smoke":"defend",
                       "ash":"attack","pulse":"defend","thermite":"attack","montagne":"attack","twitch":"attack","doc":"defend",
                       "rook":"defend","jager":"defend","bandit":"defend","blitz":"attack","iq":"attack","fuze":"attack","glaz":"attack",
                       "kapkan":"defend"}
        self.op_name = ""
        self.current_op = None
        self.get_operator()
        
        ### ROUND DATA ###
        self.round_lable = self.addTextField(text=f"{self.match.wins} : {self.match.loses}",row=0,column=0,columnspan=2,sticky="NSEW")
        ### BUTTONS ###
        self.addButton(text="Round Won",row=1,column=0,command=self.round_won,sticky="NSEW").config(bg="#64ad6a")
        self.addButton(text="Round Lost",row=1,column=1,command=self.round_lost,sticky="NSEW").config(bg="#dd3439")
        
        self.save_btn = self.addButton(text="save",row=3,column=0,command=self.save_match,sticky="NSEW").config(bg="#7cb9e8")
        self.load_btn = self.addButton(text="load",row=3,column=1,command=self.load_match,sticky="NSEW").config(bg="#3b43d6",fg="white")

    
    def get_operator(self):
        #save the previous operators data
        if self.current_op != None:
            fp = open(f"{self.op_name}.obj","wb")
            pickle.dump(self.current_op,fp)
            fp.close()
            
        self.op_name = self.prompterBox(title="Operator",promptString="Pick an operator",inputText="",fieldWidth=20).lower()
        
        #if the operator has been used before, read from the file
        try:
            fp = open(f"{self.op_name}.obj","rb")
            self.current_op = pickle.load(fp)
            fp.close()
        #no file exists so create a new Operator object and write it to a new .obj file
        except:
            self.current_op = Operator(self.op_name,self.all_ops[self.op_name])
            fp  = open(f"{self.op_name}.obj","wb")
            pickle.dump(self.current_op,fp)
            fp.close()
            
        #add the operator icon to a label
        self.img = ImageTk.PhotoImage(Image.open(f"./r6/{self.op_name}.png").resize((300,300)))
        self.img_label.configure(image=self.img)
        self.img_label.image = self.img
        
        #update the display to show new operator data
        self.update_display()

    #Show the current operators all time win/loss
    def update_display(self):
        self.display.setText(f"{self.current_op.name}, Wins: {self.current_op.wins}, Loses: {self.current_op.loses}")

    #Save the current matches round wins and loses to a .obj file
    def save_match(self):
        fp = open("frames.obj",'wb')
        pickle.dump(self.match,fp)
        fp.close()
        self.destroy()
        
    #Load in the saved match data
    def load_match(self):
        fp = open("frames.obj","rb")
        self.match = pickle.load(fp)
        fp.close()
        self.round_lable.setText(f"{self.match.wins} : {self.match.loses}")
        self.update_operator()
        self.update_display()

    #Returns True when specific round requirements are met
    def check_win(self):
        if self.match.wins == 5 or (self.match.wins == 4 and self.match.loses < 3):
            return True
        return False
    
    #Returns True when specific round requirements are met
    def check_loss(self):
        if self.match.loses == 5 or (self.match.loses == 4 and self.match.wins < 3):
            return True
        return False
    
    #Update current rounds won and check for win condition
    def round_won(self):
        self.match.wins += 1
        self.current_op.wins += 1
        self.round_lable.setText(f"{self.match.wins} : {self.match.loses}")
        self.update_display()
        if self.check_win():
            print("You won the game!")
            self.destroy()
        self.get_operator()
        
    #UPdate current rounds lost and check for lost condition
    def round_lost(self):
        self.match.loses += 1
        self.current_op.loses += 1
        self.round_lable.setText(f"{self.match.wins} : {self.match.loses}")
        self.update_display()
        if self.check_loss():
            print("You lost the game!")
            self.destroy
        self.get_operator()

In [80]:

#Base Application Window
root = tk.Tk()

###Create an object from class MainFrame
main_window = MainFrame()
main_window.grid(row=0,column=0)

match_window = None

def start_match():
    global match_window
    match_window = Match()
    match_window.grid(row=0,column=0,sticky="NSEW")

def finish_match():
    match_window.grid_forget()

#Associate the start_btn with the main windows
start_btn = tk.Button(main_window,text="Start Match",command=start_match)
start_btn.grid(row=0,column=0)
root.mainloop()